In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "kbhdp",
                             "reference": "nawi",
                             "scenario": "ps_foa"}

wt.case_study_trains.source_water = {"case_study": "kbhdp", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "kbhdp_brackish_ave"}

m = wt.case_study_trains.get_case_study(m=m)

kbhdp

------- Adding Unit Processes -------
anion_exchange
municipal_drinking
-------------------------------------

adding source

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
m.fs.kbhdp_brackish_ave.flow_vol_in.fix(0.0043812636 / 0.87991)
m.fs.kbhdp_brackish_ave.conc_mass_in[0, "tds"].fix(8.561)
#m.fs.anion_exchange.water_recovery.fix(1 - (12/100))
wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


ANION EXCHANGE:

	total cap investment: 0.15338
	cat and chem cost: 0.0
	electricity cost: 0.04088
	total fixed op cost: 0.00204

MUNICIPAL DRINKING:

	total cap investment: 0.01976
	cat and chem cost: 0.0
	electricity cost: 0.00233
	total fixed op cost: 0.00026

----------------------------------------------------------------------


----------------------------------------------------------------------
------------------- System Level Metrics and Costs -------------------
Total Capital Investment ($MM) 0.173
Annual Fixed Operating Cost ($MM/yr) 0.002
Annual Catalysts and Chemicals Cost ($MM/yr) 0.0
Annual Electricity Costs ($MM/yr) 0.043
Annual Other Variable Costs ($MM/yr) 0.0
Annual Operating Costs ($MM/yr) 0.046
Treated water (m3/s) ---> 0.004
Total w

In [4]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
None


In [10]:
0.003855511968 * 2.282e7

87982.78310976

In [4]:
m.fs.anion_exchange.display()

Block fs.anion_exchange

  Variables:
    flow_vol_in : Volumetric flowrate of water into unit
        Size=1, Index=fs.time
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
        0.0 : 1e-08 : 0.004979217874555353 : 100.0 : False : False : NonNegativeReals
    conc_mass_in : Mass concentration of species at inlet
        Size=1, Index=fs.anion_exchange.conc_mass_in_index
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        (0.0, 'tds') :  None : 8.561 :  None : False : False :  Reals
    temperature_in : Temperature at inlet
        Size=1, Index=fs.time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :  None : 300.0 :  None : False : False :  Reals
    pressure_in : Pressure at inlet
        Size=1, Index=fs.time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :     0 :   1.0 :  None : False : False : NonNegativeReals
    flow_vol_out : Volumetric flowrate of water out of unit
    

In [5]:
12780 * (29.21 / 1000) / 1000 #gr = 373.3 Kgr of additional cations a

373.30379999999997

In [4]:
m.fs.anion_exchange.conc_mass_waste[0, "tds"]()

49.500000000000014

In [6]:
m.fs.anion_exchange.flow_vol_waste[0]() * 2.282e7

12873.563758925806

In [11]:
x1 = m.fs.anion_exchange.conc_mass_waste[0, "tds"]() * m.fs.anion_exchange.flow_vol_waste[0]() #kg/s ->
x2 = 17.12 * 1000 * x1

In [12]:
x1

1.1246107046806881e-07

In [13]:
x2

0.001925333526413338

In [4]:
m.fs.anion_exchange.mass_removed[0]()

1.0

In [9]:
m.fs.anion_exchange.water_recovery.fix(0.95)

In [4]:
m.fs.anion_exchange.removal_fraction[0, "tds"]()

0.99

In [4]:
m.fs.anion_exchange.water_recovery[0]()

0.9505042512971633

In [4]:
m.fs.anion_exchange.water_recovery[0]()

0.9999998565565833

In [6]:
m.fs.anion_exchange.display()

Block fs.anion_exchange

  Variables:
    flow_vol_in : Volumetric flowrate of water into unit
        Size=1, Index=fs.time
        Key : Lower : Value    : Upper : Fixed : Stale : Domain
        0.0 : 1e-08 : 1.336278 : 100.0 : False : False : NonNegativeReals
    conc_mass_in : Mass concentration of species at inlet
        Size=1, Index=fs.anion_exchange.conc_mass_in_index
        Key          : Lower : Value                 : Upper : Fixed : Stale : Domain
        (0.0, 'tds') :  None : 5.670949272155659e-10 :  None : False : False :  Reals
    temperature_in : Temperature at inlet
        Size=1, Index=fs.time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :  None : 300.0 :  None : False : False :  Reals
    pressure_in : Pressure at inlet
        Size=1, Index=fs.time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :     0 :   1.0 :  None : False : False : NonNegativeReals
    flow_vol_out : Volumetric flowrate of water out of u

In [3]:
### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=False)

m = wt.set_bounds(m)

wt.print_ro_results(m)

Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 5.46165
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.07273

ANTI SCALANT ADDITION:

	total cap in

In [4]:
1.33*0.7

0.9309999999999999

In [4]:
m.fs.ro_first_pass.flow_vol_out[0]()

0.22106017645160778

In [5]:
m.fs.ro_first_pass.conc_mass_in[0, "tds"]()

2.5031532747942373

In [6]:
# If you need the system recovery to match better.... set a maximum recovery rate.
# The previous system recovery results must be greater than the limit you set below. If this is not the case, then
# you need to ease the bounds on the flux assumpations.

desired_recovery = 0.83

if m.fs.costing.system_recovery() > desired_recovery:
    m.fs.recovery_bound = Constraint(expr=m.fs.costing.system_recovery <= desired_recovery)
    
    wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

    wt.print_ro_results(m)

else:
    print("system recovery already lower than desired recovery. desired:", desired_recovery, 
          "current:", m.fs.costing.system_recovery())

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 5.46165
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.07273

ANTI SCALANT ADDITION:

	total cap investment: 0.09493
	cat and chem cost: 0.41919
	electricity cost: 0.0
	total fixed op cost: 0.00126

ANION EXCHANGE:

	total cap investment: 0.76905
	cat and chem cost: 0.0
	electricity cost: 0.04088
	total fixed op cost: 0.01025

RO FIRST PASS:

	total cap investment: 1.78366
	cat and chem cost: 0.01696
	electricity cost: 0.15649
	total fixed op cost: 0.02375

RO FIRST STAGE:

	total cap investment: 0.0062
	cat and chem cost: 6e-05
	electricity cost: 0.0
	total fixed op cost

In [7]:
m = wt.run_ro_no_freedom(m, skip_small=False)

kbhdp

------- Adding Unit Processes -------
well_field
media_filtration
anti_scalant_addition
anion_exchange
ro_first_pass
ro_first_stage
caustic_soda_addition
chlorination
treated_storage
deep_well_injection
municipal_drinking
surface_discharge
-------------------------------------

adding source
adding splitter
params into splitter --> [0.393, 0.607]
adding splitter

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

Flow is relatively small (< 1 m3/s). Running model with larger dummy flows to initialize...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Model finished running to initialize conditions. Now running with actual flow...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optim

In [8]:
resin_type_list = ['styrenic_gel_1', 'styrenic_gel_2', 'styrenic_macro_1', 
                   'styrenic_macro_2', 'polyacrylic', 'nitrate']

In [9]:
sens_df = pd.DataFrame()

m_scenario = "ps_foa"

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []
elec_int = []
resin_scenario_l = []
ix_cap_list = []
ix_onm_list = []

runs_per_scenario = 20
ix = m.fs.anion_exchange

lcow_list.append(value(m.fs.costing.LCOW))
water_recovery_list.append(value(m.fs.costing.system_recovery))
scenario_value.append("n/a")
scenario_name.append("baseline")
elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
elec_int.append(value(m.fs.costing.electricity_intensity))
resin_scenario_l.append("default")
ix_cap_list.append(ix.costing.fixed_cap_inv())
ix_onm_list.append(ix.costing.annual_op_main_cost())

def save_results():
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    elec_int.append(value(m.fs.costing.electricity_intensity))
    resin_scenario_l.append(resin_name)
    ix_cap_list.append(ix.costing.fixed_cap_inv())
    ix_onm_list.append(ix.costing.annual_op_main_cost())

for resin_name in resin_type_list:
    ix.resin_cost.fix(ix.resin_dict[resin_name])
    print("-------", resin_name, "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())

    ############ onstream_factor 70-100% ############
    stash_value = m.fs.costing_param.plant_cap_utilization
    scenario = "Plant Capacity Utilization 70-100%"
    print("-------", scenario, "-------")
    ub = 1
    lb = 0.7
    step = (ub - lb) / runs_per_scenario
    
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.plant_cap_utilization = i
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())
        
        save_results()
        
    m.fs.costing_param.plant_cap_utilization = stash_value  
    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    ############ onstream_factor 70-100% ############
    stash_value = m.fs.anion_exchange.ebct
    scenario = "EBCT 2-7"
    print("-------", scenario, "-------")
    ub = 7
    lb = 2
    step = (ub - lb) / runs_per_scenario
    
    for i in np.arange(lb, ub + step, step):
        ix.ebct.fix(i)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results()
    
    ix.ebct = stash_value    
    ############################################################  
    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ inlet flow +-30% ############
    stash_value = []
    for key in m.fs.flow_in_dict:   
        stash_value.append(value(getattr(m.fs, key).flow_vol_in[0]))
    scenario = "Inlet Flow +-25%"
    print("-------", scenario, "-------")
    ub = 1.3
    lb = 0.7
    step = (ub - lb) / runs_per_scenario

    for i in np.arange(lb, ub + step, step):
        q=0
        sensitivity_flow = {}
        for key in m.fs.flow_in_dict:
            getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q] * i)
            sensitivity_flow[key] = stash_value[q] * i
            q = q + 1

        wt.run_water_tap(m = m, objective=False, skip_small = True, sensitivity_flow = sensitivity_flow)
        print(scenario, stash_value[q-1] * i, "LCOW -->", m.fs.costing.LCOW())

        save_results()
        
    q=0
    for key in m.fs.flow_in_dict:    
        getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q])
        q = q + 1

    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ lifetime years ############

    stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
    scenario = "Plant Lifetime 15-45 yrs"
    print("-------", scenario, "-------")
    ub = 45
    lb = 15
    step = (ub - lb) / runs_per_scenario
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.plant_lifetime_yrs = i 
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results()
        
    m.fs.costing_param.plant_lifetime_yrs = stash_value
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ elec cost +-30% ############

    stash_value = value(m.fs.costing_param.electricity_price)
    scenario = "electricity price +- 30%"
    print("-------", scenario, "-------")
    ub = stash_value * 1.3
    lb = stash_value * 0.7
    step = (ub - lb) / runs_per_scenario
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.electricity_price = i 
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results()
        
    m.fs.costing_param.electricity_price = stash_value
    ############################################################    
    

------- styrenic_gel_1 -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.228279020345677
------- Plant Capacity Utilization 70-100% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.7 LCOW --> 0.3261128862081101
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.715 LCOW --> 0.319271357126821
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

-----------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 4.0 LCOW --> 0.2299880218553014
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 4.25 LCOW --> 0.23020164704400436
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 4.5 LCOW --> 0.2304152722327075
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 4.75 LCOW --> 0.23062889742141046
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.6970730600000008 LCOW --> 0.22072803229613783
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.7371614000000009 LCOW --> 0.21962926689126738
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.23255152411973784
------- Plant Lifetime 15-45 yrs -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 15.0 LCOW --> 0.3058121425708243
----------------------------------

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04959499999999998 LCOW --> 0.23012508238797763
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.05122999999999998 LCOW --> 0.2309338962985643
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.052864999999999975 LCOW --> 0.23174271020915113
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

---------------------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9250000000000002 LCOW --> 0.251738593891453
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9400000000000002 LCOW --> 0.24772148866978078
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9550000000000002 LCOW --> 0.24383057523517693
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.9700000000000002 LCOW --> 0.24

...


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 19.5 LCOW --> 0.2724059812020758
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 21.0 LCOW --> 0.2642450259056902
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 22.5 LCOW --> 0.2572585432469896
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 24.0 LCOW --> 0.25122509809595334
----------------------------------------------------------------------

Degrees o


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.059404999999999965 LCOW --> 0.23616786574333998
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06103999999999996 LCOW --> 0.23697667965392671
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06267499999999995 LCOW --> 0.2377854935645135
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06430999999999995 LCOW --> 0.2385943074751


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.232858199349594
------- EBCT 2-7 -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 2.0 LCOW --> 0.2283666418399216
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 2.25 LCOW --> 0.22859121971540527
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
EBCT 2-7 2.5 LCOW --> 0.22881579759088885
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

------------

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.3763663400000004 LCOW --> 0.23133814876729789
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.4164546800000004 LCOW --> 0.22987773185460153
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.4565430200000005 LCOW --> 0.22847305132548312
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-25% 1.496631


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.232858199349594
------- electricity price +- 30% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.038149999999999996 LCOW --> 0.22477006024372637
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.039784999999999994 LCOW --> 0.2255788741543132
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04141999999999999 LCOW --> 0.22638768806489987
-----------------------

In [10]:
sens_df = pd.DataFrame()

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["elec_int"] =  elec_int
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["resin"] = resin_scenario_l
sens_df["ix_cap"] = ix_cap_list
sens_df["ix_onm"] = ix_onm_list

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100

sens_df.to_csv("results/case_studies/%s_%s_sensitivity.csv" % ("KB_FOA", m_scenario))